# ASL Gesture Recognition Model Training

This notebook trains a machine learning model to classify ASL gestures using hand landmarks from Mediapipe.


In [ ]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Load training data
with open('../data/raw/combined_dataset.json', 'r') as f:
    data = json.load(f)

print(f"Loaded {len(data)} samples")
print(f"Gestures: {set([sample['gesture'] for sample in data])}")


In [ ]:
def extract_features(landmarks):
    """Extract features from hand landmarks"""
    landmarks = np.array(landmarks)
    
    # Calculate distances between key points
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    middle_tip = landmarks[12]
    ring_tip = landmarks[16]
    pinky_tip = landmarks[20]
    
    # Calculate distances
    distances = []
    key_points = [thumb_tip, index_tip, middle_tip, ring_tip, pinky_tip]
    
    for i in range(len(key_points)):
        for j in range(i + 1, len(key_points)):
            dist = np.linalg.norm(key_points[i] - key_points[j])
            distances.append(dist)
    
    # Calculate angles
    angles = []
    for i in range(len(key_points) - 2):
        v1 = key_points[i] - key_points[i + 1]
        v2 = key_points[i + 1] - key_points[i + 2]
        angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
        angles.append(angle)
    
    # Combine features
    features = np.concatenate([distances, angles])
    
    return features

# Extract features and labels
X = []
y = []

for sample in data:
    features = extract_features(sample['landmarks'])
    X.append(features)
    y.append(sample['gesture'])

X = np.array(X)
y = np.array(y)

print(f"Feature matrix shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Unique labels: {np.unique(y)}")


In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")


In [ ]:
# Train multiple models and compare
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='rbf', random_state=42),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(100, 50), random_state=42)
}

results = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)
    
    # Predictions
    y_pred = model.predict(X_test)
    
    # Calculate accuracy
    accuracy = model.score(X_test, y_test)
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'predictions': y_pred
    }
    
    print(f"{name} Accuracy: {accuracy:.3f}")
    
    # Classification report
    print(f"\n{name} Classification Report:")
    print(classification_report(y_test, y_pred))


In [ ]:
# Save the best model
best_model_name = max(results.keys(), key=lambda k: results[k]['accuracy'])
best_model = results[best_model_name]['model']

print(f"\nBest model: {best_model_name} with accuracy: {results[best_model_name]['accuracy']:.3f}")

# Save model
model_path = '../saved_models/asl_gesture_classifier.joblib'
joblib.dump(best_model, model_path)
print(f"Model saved to: {model_path}")

# Save feature extractor function
import pickle
with open('../saved_models/feature_extractor.pkl', 'wb') as f:
    pickle.dump(extract_features, f)
print("Feature extractor saved to: ../saved_models/feature_extractor.pkl")
